In [1]:
''''
В данном задании предлагается реализовать простейший MapReduce над корпусом текстов — нужно выполнить предобработку текстов и подсчет количества вхождений слов (и реализовать многопроцессно!)
'''

"'\nВ данном задании предлагается реализовать простейший MapReduce над корпусом текстов — нужно выполнить предобработку текстов и подсчет количества вхождений слов (и реализовать многопроцессно!)\n"

In [2]:
# !pip install scikit-learn nltk

In [12]:
import re

from typing import List

import nltk
import time
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords as nltk_stop
nltk.download("stopwords")

from sklearn.datasets import fetch_20newsgroups

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\almta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
stopwords = set(nltk_stop.words("english"))
list(stopwords)[:5]

['itself', 'mightn', 'this', 'so', "won't"]

In [14]:
from sklearn.datasets import fetch_20newsgroups

news = fetch_20newsgroups().data * 10
len(news)

113140

In [15]:
''''Подсчёт в одном процессе'''
def tokenize(text: str) -> List[str]:
    tokenizer = RegexpTokenizer(r"\w+")
    return tokenizer.tokenize(text)


def clear_tokens(tokens: List[str]) -> List[str]:
    return [re.sub(r"[^a-z]", "", token) for token in tokens]


def is_token_informative(token: str) -> bool:
    return token and token not in stopwords and len(token) > 2


def count_words(texts: List[str]):
    tokenizer = RegexpTokenizer(r"\w+")
    cnt = Counter()
    for text in texts:
        tokens = tokenize(text.lower())
        tokens_clear = clear_tokens(tokens)
        informative_tokens = filter(is_token_informative, tokens_clear)
        cnt.update(informative_tokens)
    return cnt

In [16]:
%%time
cnt = count_words(news)
cnt.most_common(10)

NameError: name 'Counter' is not defined

In [10]:
'''
Подсчёт с помощью MapReduce
'''

import math
from typing import Dict, List, Sequence


# пригодится для партицирования данных
def chunkify(data: Sequence, n_chunks: int) -> List:
    chunk_size = math.ceil(len(data) / n_chunks)
    return [data[i * chunk_size: (i + 1) * chunk_size] for i in range(n_chunks)]


def words_count_map(text: str) -> Dict[str, int]:
    tokens = tokenize(text.lower())
    tokens_clear = clear_tokens(tokens)
    informative_tokens = filter(is_token_informative, tokens_clear)
    return Counter(informative_tokens)


def words_count_reduce(cnt1: Dict[str, int], cnt2: Dict[str, int]) -> Dict[str, int]:
    cnt1.update(cnt2)
    return cnt1


def words_count_chunk_mapper(chunk: List[str]) -> Dict[str, int]:
    mapped = map(words_count_map, chunk)
    reduced = reduce(words_count_reduce, mapped)
    return reduced


In [17]:
from multiprocessing import Pool
from functools import reduce

n_parts = 40
parts = chunkify(news, n_parts)

In [ ]:
%%time
n_workers = 8

with Pool(n_workers) as pool:
    mapped = pool.map(words_count_chunk_mapper, parts)
    reduced = reduce(words_count_reduce, mapped, Counter())


reduced.most_common(10)